In [1]:
!pip install -q transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 10.1 MB/s eta 0:00:00


In [2]:
!pip install pyarrow

In [3]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00


In [4]:
!pip -q install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00


## Load data

In [1]:
import wandb

In [2]:
from urllib import request
import pandas as pd
from collections import Counter
import random
import numpy as np
import pandas as pd
import copy
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

In [3]:
import datasets
from datasets import Dataset
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer

In [4]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [5]:
from dont_patronize_me import DontPatronizeMe

In [6]:
dpm = DontPatronizeMe('.', 'task4_test.tsv')

In [7]:
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [8]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [9]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

In [10]:
data=dpm.train_task1_df
data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [11]:
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

trdf1 = pd.DataFrame(rows)
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [12]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })
tedf1 = pd.DataFrame(rows)
tedf1

,par_id,community,text,label
0,4046,hopeless,We also know that they can benefit by receivin...,1
1,1279,refugee,Pope Francis washed and kissed the feet of Mus...,1
2,8330,refugee,Many refugees do n't want to be resettled anyw...,1
3,4063,in-need,"""Budding chefs , like """" Fred """" , """" Winston ...",1
4,4089,homeless,"""In a 90-degree view of his constituency , one...",1
...,...,...,...,...
2089,10462,homeless,"The sad spectacle , which occurred on Saturday...",0
2090,10463,refugee,""""""" The Pakistani police came to our house and...",0
2091,10464,disabled,"""When Marie O'Donoghue went looking for a spec...",0
2092,10465,women,"""Sri Lankan norms and culture inhibit women fr...",0


In [13]:
rows_list = tedf1.values.tolist()

# Shuffle the list using random.shuffle
random.shuffle(rows_list)

# Create a new DataFrame from the shuffled list
tedf1 = pd.DataFrame(rows_list, columns=tedf1.columns)
tedf1

,par_id,community,text,label
0,10125,refugee,"""Maimane also complained about the state of th...",0
1,8447,refugee,"She further argued that , the then Mahama-led ...",0
2,10350,women,"According to the SCMP , the unions are demandi...",0
3,10383,vulnerable,UN agencies are providing food and tents to th...,0
4,9914,women,This was revealed by KWIECO Managing Director ...,0
...,...,...,...,...
2089,9348,poor-families,"""Tight money prompted his university to launch...",0
2090,9872,vulnerable,This caf ? will serve as a platform for buildi...,0
2091,9633,hopeless,"But Joan was happier than she had ever been , ...",0
2092,10414,poor-families,Human remains in the former Walker 's Croft bu...,0


In [14]:
import datasets
from datasets import Dataset
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer

In [15]:
# Split the dataset into train and test sets
train_df, val_df = train_test_split(trdf1, test_size=0.2, random_state=42)

train_dataset =  Dataset.from_dict({"data": train_df["text"], "label": train_df["label"]}).with_format("torch")
val_dataset =  Dataset.from_dict({"data": val_df["text"], "label": val_df["label"]}).with_format("torch")
test_dataset =  Dataset.from_dict({"data": tedf1["text"], "label": tedf1["label"]}).with_format("torch")

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

6700
1675
2094


In [16]:
def tokenize_function(example):
    return tokenizer(example["data"], truncation=True)


In [17]:
from sklearn.metrics import accuracy_score, f1_score

# Define evaluation metrics function
def compute_metrics(p):
    predictions = p.predictions.argmax(axis=1)
    labels = p.label_ids

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='binary')


    # return {"accuracy": accuracy, "f1": f1, "predictions": list(predictions)}
    return {"accuracy": accuracy, "f1": f1}

In [18]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

## Roberta Hate speech

In [19]:
# Load model

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [20]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/6700 [00:00<?, ? examples/s]

Map:   0%|          | 0/1675 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

In [21]:
batch_size = 16

training_args = TrainingArguments("train_logs", evaluation_strategy="epoch",\
                                  learning_rate=2e-6,
                                  num_train_epochs=10,
                                  lr_scheduler_type="cosine",
                                  optim = "adamw_torch",
                                  weight_decay=0,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  run_name="roberta-base-hate-best",  # name of the W&B run (optional)
                                  logging_steps=1,  # how often to log to W&B
                                  )


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [22]:
trainer.train()

wandb: Currently logged in as: hfp. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.057000,0.259912,0.894328,0.043243
2,0.080400,0.236954,0.905672,0.455172
3,0.194100,0.235244,0.909254,0.536585
4,0.273600,0.258228,0.915224,0.489209
5,0.030800,0.258911,0.911642,0.546012
6,0.028500,0.287335,0.906866,0.521472
7,0.009800,0.295311,0.909254,0.527950
8,0.020700,0.311492,0.914030,0.526316
9,0.023700,0.313397,0.909851,0.517572
10,0.002500,0.314614,0.907463,0.514107


Checkpoint destination directory train_logs/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-3000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-3500 already exists and is non-empty.Saving will proceed but saved res

TrainOutput(global_step=4190, training_loss=0.17364659802076854, metrics={'train_runtime': 589.3556, 'train_samples_per_second': 113.683, 'train_steps_per_second': 7.109, 'total_flos': 4433028671788080.0, 'train_loss': 0.17364659802076854, 'epoch': 10.0})

In [23]:
predictions_val_roberta = trainer.predict(tokenized_val_dataset).predictions
print(predictions_val_roberta.shape)
predictions_val_roberta_lables = predictions_val_roberta.argmax(axis=1)
predictions_val_roberta_lables

(1675, 2)


array([0, 0, 0, ..., 0, 0, 0])

In [24]:
labels2file([[k] for k in predictions_val_roberta_lables], 'val_roberta.txt')

In [25]:
trainer.predict(tokenized_val_dataset).metrics

{'test_loss': 0.3146141469478607,
 'test_accuracy': 0.9074626865671642,
 'test_f1': 0.5141065830721003,
 'test_runtime': 3.7973,
 'test_samples_per_second': 441.104,
 'test_steps_per_second': 27.651}

In [26]:
results = trainer.evaluate(tokenized_val_dataset)

print(results)

{'eval_loss': 0.3146141469478607, 'eval_accuracy': 0.9074626865671642, 'eval_f1': 0.5141065830721003, 'eval_runtime': 3.8134, 'eval_samples_per_second': 439.245, 'eval_steps_per_second': 27.535, 'epoch': 10.0}


In [27]:
predictions_test_roberta = trainer.predict(tokenized_test_dataset)
print(predictions_test_roberta.metrics)

predictions_test_roberta_labels = predictions_test_roberta.predictions.argmax(axis=1)
predictions_test_roberta

{'test_loss': 0.26160696148872375, 'test_accuracy': 0.9245463228271251, 'test_f1': 0.5927835051546393, 'test_runtime': 4.9766, 'test_samples_per_second': 420.769, 'test_steps_per_second': 26.323}


PredictionOutput(predictions=array([[ 3.3578987 , -3.2673616 ],
       [-1.323425  ,  0.81386113],
       [ 3.060962  , -3.0064576 ],
       ...,
       [ 2.4528756 , -2.5364082 ],
       [ 3.0025208 , -2.853967  ],
       [ 3.6041152 , -3.3718195 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.26160696148872375, 'test_accuracy': 0.9245463228271251, 'test_f1': 0.5927835051546393, 'test_runtime': 4.9766, 'test_samples_per_second': 420.769, 'test_steps_per_second': 26.323})

In [28]:
labels2file([[k] for k in predictions_test_roberta_labels], 'dev_roberta.txt')

In [29]:
Counter(predictions_test_roberta_labels)

Counter({0: 1905, 1: 189})

In [30]:
dpm.load_test()
test_df = dpm.test_set_df

In [31]:
test_df

,par_id,art_id,keyword,country,text
0,t_0,@@7258997,vulnerable,us,"In the meantime , conservatives are working to..."
1,t_1,@@16397324,women,pk,In most poor households with no education chil...
2,t_2,@@16257812,migrant,ca,The real question is not whether immigration i...
3,t_3,@@3509652,migrant,gb,"In total , the country 's immigrant population..."
4,t_4,@@477506,vulnerable,ca,"Members of the church , which is part of Ken C..."
...,...,...,...,...,...
3827,t_3893,@@20319448,migrant,jm,In a letter dated Thursday to European Commiss...
3828,t_3894,@@9990672,poor-families,au,They discovered that poor families with health...
3829,t_3895,@@37984,migrant,ca,"She married at 19 , to Milan ( Emil ) Badovina..."
3830,t_3896,@@9691377,immigrant,us,The United Kingdom is n't going to devolve int...


In [32]:
official_test_dataset =  Dataset.from_dict({"data": test_df["text"]}).with_format("torch")

In [33]:

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
tokenized_official_test_dataset = official_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3832 [00:00<?, ? examples/s]

In [34]:
predictions = trainer.predict(tokenized_official_test_dataset)
print(predictions.metrics)

predictions = predictions.predictions.argmax(axis=1)
predictions

{'test_runtime': 9.0379, 'test_samples_per_second': 423.991, 'test_steps_per_second': 26.555}


array([0, 0, 0, ..., 0, 0, 0])

In [35]:
labels2file([[k] for k in predictions], 'test_roberta.txt')

## Deberta

In [19]:
from transformers import DebertaForSequenceClassification, DebertaTokenizer, AdamW
import torch

In [20]:
model_name = 'microsoft/deberta-base'
model = DebertaForSequenceClassification.from_pretrained(model_name)
tokenizer = DebertaTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaForSequenceClassification were not initialized from the model chec

In [21]:
def tokenize_function(example):
    return tokenizer(example["data"], truncation=True)


tokenized_train_dataset_De = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset_De = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset_De = test_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/6700 [00:00<?, ? examples/s]

Map:   0%|          | 0/1675 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

In [22]:
batch_size = 8

training_args1 = TrainingArguments("train_logs", evaluation_strategy="epoch",\
                                  learning_rate=2e-6,
                                  num_train_epochs=10,
                                  lr_scheduler_type="cosine",
                                  optim = "adamw_torch",
                                  weight_decay=0,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  run_name="Deberta-ms",  # name of the W&B run (optional)
                                  logging_steps=1,  # how often to log to W&B
                                                                  )


trainer = Trainer(
    model,
    training_args1,
    train_dataset=tokenized_train_dataset_De,
    eval_dataset=tokenized_val_dataset_De,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [23]:
trainer.train()

wandb: Currently logged in as: hfp. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.011400,0.257529,0.901493,0.023669
2,0.010300,0.231561,0.908060,0.518750
3,0.018600,0.267520,0.914030,0.520000
4,0.005100,0.298254,0.912836,0.503401
5,0.001600,0.337542,0.912239,0.527331
6,0.002900,0.367340,0.912239,0.514851
7,0.001400,0.410349,0.909851,0.488136
8,0.001600,0.426071,0.909851,0.491582
9,0.009500,0.433371,0.907463,0.498382
10,0.001300,0.434149,0.907463,0.501608


Checkpoint destination directory train_logs/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-3000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-3500 already exists and is non-empty.Saving will proceed but saved res

TrainOutput(global_step=8380, training_loss=0.18346322484795524, metrics={'train_runtime': 1059.0356, 'train_samples_per_second': 63.265, 'train_steps_per_second': 7.913, 'total_flos': 4356804581549088.0, 'train_loss': 0.18346322484795524, 'epoch': 10.0})

In [24]:
predictions_val_deberta = trainer.predict(tokenized_val_dataset_De).predictions
print(predictions_val_deberta.shape)
predictions_val_deberta_lables = predictions_val_deberta.argmax(axis=1)
predictions_val_deberta_lables

(1675, 2)


array([0, 0, 0, ..., 0, 0, 0])

In [25]:
trainer.predict(tokenized_val_dataset_De).metrics

{'test_loss': 0.43414929509162903,
 'test_accuracy': 0.9074626865671642,
 'test_f1': 0.5016077170418006,
 'test_runtime': 6.4334,
 'test_samples_per_second': 260.361,
 'test_steps_per_second': 32.642}

In [26]:
predictions_test_deberta = trainer.predict(tokenized_test_dataset_De)
print(predictions_test_deberta.metrics)

predictions_test_deberta_labels = predictions_test_deberta.predictions.argmax(axis=1)
predictions_test_deberta

{'test_loss': 0.3399055302143097, 'test_accuracy': 0.9312320916905444, 'test_f1': 0.6210526315789474, 'test_runtime': 8.0788, 'test_samples_per_second': 259.198, 'test_steps_per_second': 32.431}


PredictionOutput(predictions=array([[ 3.2443728, -3.8731358],
       [ 3.2330189, -3.412592 ],
       [ 3.6357996, -4.1420517],
       ...,
       [ 3.564238 , -3.883561 ],
       [ 3.4531167, -3.8740396],
       [ 3.5120053, -4.0967445]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.3399055302143097, 'test_accuracy': 0.9312320916905444, 'test_f1': 0.6210526315789474, 'test_runtime': 8.0788, 'test_samples_per_second': 259.198, 'test_steps_per_second': 32.431})

In [27]:
labels2file([[k] for k in predictions_val_deberta_lables], 'val_deberta.txt')

In [28]:
labels2file([[k] for k in predictions_test_deberta_labels], 'dev_deberta.txt')

In [33]:
tokenized_official_test_dataset = official_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3832 [00:00<?, ? examples/s]

In [34]:
predictions = trainer.predict(tokenized_official_test_dataset)
print(predictions.metrics)

predictions = predictions.predictions.argmax(axis=1)
predictions

{'test_runtime': 14.6392, 'test_samples_per_second': 261.764, 'test_steps_per_second': 32.72}


array([0, 0, 0, ..., 0, 0, 0])

In [35]:
labels2file([[k] for k in predictions], 'test_deberta.txt')

## Bert-Uncased Model

In [38]:
# Load model
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
def tokenize_function(example):
    return tokenizer(example["data"], truncation=True)


tokenized_train_dataset_Uc = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset_Uc = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset_Uc = test_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/6700 [00:00<?, ? examples/s]

Map:   0%|          | 0/1675 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

In [40]:
batch_size = 16

training_args1 = TrainingArguments("train_logs", evaluation_strategy="epoch",\
                                  learning_rate=2e-6,
                                  num_train_epochs=10,
                                  lr_scheduler_type="cosine",
                                  optim = "adamw_torch",
                                  weight_decay=0,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  run_name="Bert_UC",  # name of the W&B run (optional)
                                  logging_steps=1,  # how often to log to W&B
                                                                  )


trainer = Trainer(
    model,
    training_args1,
    train_dataset=tokenized_train_dataset_Uc,
    eval_dataset=tokenized_val_dataset_Uc,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.091100,0.302983,0.900299,0.000000
2,0.226100,0.241271,0.911045,0.251256
3,0.119300,0.226889,0.909254,0.428571
4,0.360000,0.239528,0.911045,0.371308
5,0.012800,0.237794,0.913433,0.426877
6,0.054600,0.239813,0.914627,0.452107
7,0.031600,0.244257,0.915224,0.466165
8,0.091200,0.253267,0.915821,0.459770
9,0.171500,0.251534,0.915821,0.479705
10,0.015400,0.250755,0.916418,0.489051


Checkpoint destination directory train_logs/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-3000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train_logs/checkpoint-3500 already exists and is non-empty.Saving will proceed but saved res

TrainOutput(global_step=4190, training_loss=0.20180806658055125, metrics={'train_runtime': 578.6838, 'train_samples_per_second': 115.78, 'train_steps_per_second': 7.241, 'total_flos': 4518597320323440.0, 'train_loss': 0.20180806658055125, 'epoch': 10.0})

In [42]:
predictions_val_uc = trainer.predict(tokenized_val_dataset_Uc).predictions
print(predictions_val_uc.shape)
predictions_val_uc_lables = predictions_val_uc.argmax(axis=1)
predictions_val_uc_lables

(1675, 2)


array([0, 0, 0, ..., 0, 0, 0])

In [43]:
trainer.predict(tokenized_val_dataset_Uc).metrics

{'test_loss': 0.25075486302375793,
 'test_accuracy': 0.9164179104477612,
 'test_f1': 0.48905109489051096,
 'test_runtime': 3.9772,
 'test_samples_per_second': 421.15,
 'test_steps_per_second': 26.4}

In [44]:
predictions_test_uc = trainer.predict(tokenized_test_dataset_Uc)
print(predictions_test_uc.metrics)

predictions_test_uc_labels = predictions_test_uc.predictions.argmax(axis=1)
predictions_test_uc

{'test_loss': 0.23365315794944763, 'test_accuracy': 0.9173829990448902, 'test_f1': 0.46105919003115264, 'test_runtime': 5.0219, 'test_samples_per_second': 416.97, 'test_steps_per_second': 26.086}


PredictionOutput(predictions=array([[ 2.368462  , -2.616525  ],
       [ 0.6320533 , -0.0291136 ],
       [ 0.91434115, -0.91911733],
       ...,
       [ 2.0118618 , -2.2450657 ],
       [ 2.3357527 , -3.0234153 ],
       [ 2.230428  , -2.90201   ]], dtype=float32), label_ids=array([0, 0, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.23365315794944763, 'test_accuracy': 0.9173829990448902, 'test_f1': 0.46105919003115264, 'test_runtime': 5.0219, 'test_samples_per_second': 416.97, 'test_steps_per_second': 26.086})

In [45]:
labels2file([[k] for k in predictions_val_uc_lables], 'val_uc.txt')

In [46]:
labels2file([[k] for k in predictions_test_uc_labels], 'dev_uc.txt')

In [51]:
tokenized_official_test_dataset = official_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3832 [00:00<?, ? examples/s]

In [52]:
predictions = trainer.predict(tokenized_official_test_dataset)
print(predictions.metrics)

predictions = predictions.predictions.argmax(axis=1)
predictions

{'test_runtime': 9.4289, 'test_samples_per_second': 406.411, 'test_steps_per_second': 25.454}


array([0, 0, 0, ..., 0, 0, 0])

In [53]:
labels2file([[k] for k in predictions], 'test_uc.txt')

## Ensemble Scores

In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Load labels from text files
with open('val_uc.txt', 'r') as file:
    labels_model1 = [int(line.strip()) for line in file]

with open('val_roberta.txt', 'r') as file:
    labels_model2 = [int(line.strip()) for line in file]

with open('val_deberta.txt', 'r') as file:
    labels_model3 = [int(line.strip()) for line in file]


labels_model1= val_dataset['label']

# Hard voting ensemble
ensemble_predictions = np.array([np.argmax(np.bincount(votes)) for votes in zip(labels_model1, labels_model2, labels_model3)])


accuracy = accuracy_score(labels_model1, ensemble_predictions)
precision = precision_score(labels_model1, ensemble_predictions)
recall = recall_score(labels_model1, ensemble_predictions)
f1 = f1_score(labels_model1, ensemble_predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.9361
Precision: 0.7239
Recall: 0.5808
F1 Score: 0.6445


In [37]:
# Dev Set Results

with open('dev_roberta.txt', 'r') as file:
    labels_model1 = [int(line.strip()) for line in file]

with open('dev_deberta.txt', 'r') as file:
    labels_model2 = [int(line.strip()) for line in file]

with open('dev_uc.txt', 'r') as file:
    labels_model3 = [int(line.strip()) for line in file]


labels_model1= test_dataset['label']

# Hard voting ensemble
ensemble_predictions = np.array([np.argmax(np.bincount(votes)) for votes in zip(labels_model1, labels_model2, labels_model3)])

accuracy = accuracy_score(labels_model1, ensemble_predictions)
precision = precision_score(labels_model1, ensemble_predictions)
recall = recall_score(labels_model1, ensemble_predictions)
f1 = f1_score(labels_model1, ensemble_predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.9618
Precision: 0.9760
Recall: 0.6131
F1 Score: 0.7531


In [38]:
ensemble_predictions

array([0, 0, 0, ..., 0, 0, 0])

In [39]:
labels2file([[k] for k in ensemble_predictions], 'dev.txt')

In [45]:
# Test Set Results

with open('test_roberta.txt', 'r') as file:
    labels_model1 = [int(line.strip()) for line in file]

with open('test_deberta.txt', 'r') as file:
    labels_model2 = [int(line.strip()) for line in file]

with open('test_uc.txt', 'r') as file:
    labels_model3 = [int(line.strip()) for line in file]

# Hard voting ensemble
ensemble_predictions = np.array([np.argmax(np.bincount(votes)) for votes in zip(labels_model1, labels_model2, labels_model3)])



In [46]:
ensemble_predictions.shape

(3832,)

In [47]:
ensemble_predictions

array([0, 0, 0, ..., 0, 0, 0])

In [48]:
labels2file([[k] for k in ensemble_predictions], 'test.txt')